In [85]:
#pip install --upgrade google-cloud-bigquery
#!pip install openpyxl
#!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
#!pip install pyarrow


In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import openpyxl
from openpyxl.chart import BarChart, Reference, Series
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText 
from email import encoders
import smtplib

def buildWorkbook(df_notstarted, df_notmoved, period, sheet):
    
    #List trackers and VINs rows for period
    rows = [
        ('Not started in ' + period, '', '', '', 'Not moved in ' + period, '', '', '', len(df_notstarted), len(df_notmoved)),
        ('VIN', 'TrackerId', '' , '', 'VIN', 'TrackerId')
    ]
    if len(df_notstarted) > len(df_notmoved):
        for i in range(len(df_notstarted)):
            if(i >= len(df_notmoved)):
                rows.append([df_notstarted.iloc[i].vin, df_notstarted.iloc[i].trackerId])
            else:
                rows.append([df_notstarted.iloc[i].vin, df_notstarted.iloc[i].trackerId, 
                '', '', df_notmoved.iloc[i].vin, df_notmoved.iloc[i].trackerId])
    else:
        for i in range(len(df_notmoved)):
            if(i >= len(df_notstarted)):
                rows.append(['', '', '', '', df_notmoved.iloc[i].vin, df_notmoved.iloc[i].trackerId])
            else:
                rows.append([df_notstarted.iloc[i].vin, df_notstarted.iloc[i].trackerId, 
                '', '', df_notmoved.iloc[i].vin, df_notmoved.iloc[i].trackerId])

    for row in rows:
        sheet.append(row)

    chart = BarChart()
    chart.title = period
    chart.x_axis.title = "Not started / Not moved"
    chart.y_axis.title = "Vehicles"

    # Not started series
    values_notstarted = Reference(sheet, min_col=9, min_row=1)
    series_notstarted = Series(values_notstarted, title='Not started')
    chart.append(series_notstarted)

    # Not moved series
    values_notmoved = Reference(sheet, min_col=10, min_row=1)
    series_notmoved = Series(values_notmoved, title='Not moved')
    chart.append(series_notmoved)

    labels = Reference(sheet, min_col=4, min_row=1)
    chart.set_categories(labels)
    sheet.add_chart(chart, "I5")

    return sheet


In [2]:
key_path = "C:/{local path}/us-auto-dev-d4b39c64167a.json"
cred= service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=cred, project=cred.project_id )
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=cred)

In [14]:
period_30 = "30 days"
period_45 = "45 days"
period_60 = "60 days"
period_90 = "90 days"
query_string = """SELECT * FROM `us-auto-dev.kernersville.vehicles_notmoved_or_started`"""
df = (
    client.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient))

df_notstarted_30 = df[['vin', 'trackerId']][df.notstarted_last30days == True]
df_notmoved_30 = df[['vin', 'trackerId']][df.notmoved_last30days == True]
df_notstarted_45 = df[['vin', 'trackerId']][df.notstarted_btw30and45days == True]
df_notmoved_45 = df[['vin', 'trackerId']][df.notmoved_btw30and45days == True]
df_notstarted_60 = df[['vin', 'trackerId']][df.notstarted_btw45and60days == True]
df_notmoved_60 = df[['vin', 'trackerId']][df.notmoved_btw45and60days == True]
df_notstarted_90 = df[['vin', 'trackerId']][df.notstarted_btw60and90days == True]
df_notmoved_90 = df[['vin', 'trackerId']][df.notmoved_btw60and90days == True]

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.title = period_30
buildWorkbook(df_notstarted_30, df_notmoved_30, period_30, sheet1)
sheet2 = wb.create_sheet(period_45)
buildWorkbook(df_notstarted_45, df_notmoved_45, period_45, sheet2)
sheet3 = wb.create_sheet(period_60)
buildWorkbook(df_notstarted_60, df_notmoved_60, period_60, sheet3)
sheet4 = wb.create_sheet(period_90)
buildWorkbook(df_notstarted_90, df_notmoved_90, period_90, sheet4)

wb.save("kernersville_metrics.xlsx")
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText 
from email import encoders
import smtplib

me = {gmailsending}
password = {password}
server = "smtp.gmail.com:587"
you = {emailreceiver}


html = """
<html><body><p>Hello, Friend.</p>
<p>Here is your data.</p>

<p>Regards,</p>
<p>Me</p>
</body></html>
"""

message = MIMEMultipart("alternative", None, [ MIMEText(html, 'html')])

message['Subject'] = "Your data"
message['From'] = me
message['To'] = you

part = MIMEBase('application', 'octet-stream')
part.set_payload(open('kernersville_metrics.xlsx', 'rb').read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="kernersville_metrics.xlsx"')
message.attach(part)


server = smtplib.SMTP(server)
server.ehlo()
server.starttls()
server.login(me, password)
server.sendmail(me, you, message.as_string())
server.quit()

(221, b'2.0.0 closing connection m18sm2409693vka.54 - gsmtp')